In [2]:
from datasets import load_dataset

/home/gokhan/miniconda3/envs/pytorch-gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
issues_dataset = load_dataset("lewtun/github-issues", split = 'train')

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [5]:
issues_dataset[0]

{'url': 'https://api.github.com/repos/huggingface/datasets/issues/2955',
 'repository_url': 'https://api.github.com/repos/huggingface/datasets',
 'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/2955/labels{/name}',
 'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/2955/comments',
 'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/2955/events',
 'html_url': 'https://github.com/huggingface/datasets/pull/2955',
 'id': 1003999469,
 'node_id': 'PR_kwDODunzps4sHuRu',
 'number': 2955,
 'title': 'Update legacy Python image for CI tests in Linux',
 'user': {'login': 'albertvillanova',
  'id': 8515462,
  'node_id': 'MDQ6VXNlcjg1MTU0NjI=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/8515462?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/albertvillanova',
  'html_url': 'https://github.com/albertvillanova',
  'followers_url': 'https://api.github.com/users/albertvillanova/followers',
  'following_u

In [6]:
issues_dataset = issues_dataset.filter(
    lambda x : (x['is_pull_request'] == False and len(x['comments']) > 0)
)
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 808
})

In [7]:
columns = issues_dataset.column_names

In [8]:
columns_we_need = ['title', 'html_url', 'comments','body']

In [9]:
columns_we_dont_need = set(columns_we_need).symmetric_difference(columns)

In [10]:
issues_dataset = issues_dataset.remove_columns(columns_we_dont_need)

In [11]:
issues_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [12]:
issues_dataset[0] # as you see the comments column contain list so it is better if we saparat it so we will use expore with pandas

{'html_url': 'https://github.com/huggingface/datasets/issues/2945',
 'title': 'Protect master branch',
 'comments': ['Cool, I think we can do both :)',
  '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).'],
 'body': 'After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of m

In [13]:
issues_dataset.set_format('pandas')
df = issues_dataset[:]

In [14]:
comments_df = df.explode('comments', ignore_index=True)

In [15]:
comments_df.head(5)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"Cool, I think we can do both :)",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...
2,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Hi ! I guess the caching mechanism should have...,## Describe the bug\r\nAfter upgrading to data...
3,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"If it's easy enough to implement, then yes ple...",## Describe the bug\r\nAfter upgrading to data...
4,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Well it can cause issue with anyone that updat...,## Describe the bug\r\nAfter upgrading to data...


In [16]:
from datasets import Dataset
ds = Dataset.from_pandas(comments_df)

In [17]:
ds

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})

In [18]:
ds = ds.map(
    lambda x : {"comments_length" : len(x['comments'].split())}
)

Map: 100%|████████████████████████████████████████████████████████████████| 2964/2964 [00:00<00:00, 34607.99 examples/s]


In [19]:
ds_filter = ds.filter(
    lambda x : (x['comments_length']>15)
)

Filter: 100%|████████████████████████████████████████████████████████████| 2964/2964 [00:00<00:00, 177835.08 examples/s]


In [20]:
ds_filter

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comments_length'],
    num_rows: 2175
})

In [21]:
def concate_text(simple):
    return {"text" :  simple['title'] + " \n " + simple['body'] + " \n " + simple['comments']}
comments_dataset = ds_filter.map(concate_text)

Map: 100%|████████████████████████████████████████████████████████████████| 2175/2175 [00:00<00:00, 15948.29 examples/s]


In [22]:
comments_dataset['text'][0]

'Protect master branch \n After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n  - Currently, simple merge commits are already disabled\r\n  - I propose to disable rebase merging as well\r\n- ~~Protect the master branch from direct pushes (to avoid accidentally pushing of merge commits)~~\r\n  - ~~This protection would reject direct pushes to master branch~~\r\n  - ~~If so, for each release (when we need to commit directly to the master branch), we should p

In [23]:
from transformers import AutoModel, AutoTokenizer

In [24]:
import torch 
torch.cuda.is_available()

True

In [25]:
model_checkpoint = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [27]:
model = AutoModel.from_pretrained(model_checkpoint)

In [28]:
device = torch.device('cuda')

In [29]:
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [30]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:,0]
    #in here we take the cls because according to self-attention cls contain a summary of all sentence.

In [31]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [32]:
emdeding_sentence = get_embeddings(comments_dataset['text'][0])

In [33]:
emdeding_sentence.shape

torch.Size([1, 768])

In [34]:
embedding_dataset = comments_dataset.map(
    lambda x : {'embedding' : get_embeddings(x['text']).detach().cpu().numpy()[0]}
)

Map: 100%|███████████████████████████████████████████████████████████████████| 2175/2175 [01:21<00:00, 26.83 examples/s]


In [35]:
len(embedding_dataset['embedding'][0])

768

In [36]:
embedding_dataset.add_faiss_index(column='embedding')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 603.21it/s]


Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comments_length', 'text', 'embedding'],
    num_rows: 2175
})

In [37]:
question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [41]:
score, sample =  embedding_dataset.get_nearest_examples(index_name='embedding',query=question_embedding, k=5)

In [45]:
import pandas as pd

In [46]:
df = pd.DataFrame.from_dict(sample) 

In [51]:
df["scores"] = score
df.sort_values("scores", ascending=False, inplace=True)

In [62]:
def git_search(query):
    query_emb = get_embeddings([query]).cpu().detach().numpy()
    score, sample = embedding_dataset.get_nearest_examples(k= 5, index_name='embedding', query=query_emb)
    df = pd.DataFrame.from_dict(sample)
    df['scores'] = score 
    df.sort_values('scores', ascending = False, inplace = True)
    df.reset_index(inplace=True)
    return df

In [63]:
git_search("how can i solve type error")

,index,html_url,title,comments,body,comments_length,text,embedding,scores
0,4,https://github.com/huggingface/datasets/issues...,TypeError: 'NoneType' object is not callable,"> * For the platform, we need to know the oper...",## Describe the bug\r\n\r\nTypeError: 'NoneTyp...,154,TypeError: 'NoneType' object is not callable \...,"[-0.23339280486106873, -0.21219104528427124, 0...",31.064016
1,2,https://github.com/huggingface/datasets/issues...,TypeError: 'NoneType' object is not callable,One naive question: do you have internet acces...,## Describe the bug\r\n\r\nTypeError: 'NoneTyp...,16,TypeError: 'NoneType' object is not callable \...,"[-0.24667498469352722, -0.2166071981191635, -0...",30.874662
2,3,https://github.com/huggingface/datasets/issues...,TypeError: 'NoneType' object is not callable,One naive question: do you have internet acces...,## Describe the bug\r\n\r\nTypeError: 'NoneTyp...,16,TypeError: 'NoneType' object is not callable \...,"[-0.24667498469352722, -0.2166071981191635, -0...",30.874662
3,0,https://github.com/huggingface/datasets/issues...,TypeError: 'NoneType' object is not callable,"- For the platform, we need to know the operat...",## Describe the bug\r\n\r\nTypeError: 'NoneTyp...,69,TypeError: 'NoneType' object is not callable \...,"[-0.2277713567018509, -0.31935057044029236, 0....",28.649061
4,1,https://github.com/huggingface/datasets/issues...,TypeError: 'NoneType' object is not callable,"- For the platform, we need to know the operat...",## Describe the bug\r\n\r\nTypeError: 'NoneTyp...,69,TypeError: 'NoneType' object is not callable \...,"[-0.2277713567018509, -0.31935057044029236, 0....",28.649061
